In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import os

In [2]:
dataset_bik=pd.read_csv('Bik dataset - papers with endpoint reached.csv',encoding = "ISO-8859-1",sep='\t')
bik=dataset_bik.iloc[:214]
bik.head()

,Authors,Title,Citation,DOI,Year,Month,0,1,2,3,FINDINGS,Reported,Correction Date,Retraction,Correction,No Action,SUM \nCompleted
0,"Inka Regine Weingaertner, Sarah Koutnik, Herma...",Chronic Morphine Treatment Attenuates Cell Gro...,"PLOS One, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053510,2013.0,01January,1.0,NaN,1.0,NaN,"duplicated blot, shifted microscopy",1,10/09/2015,NaN,1.0,NaN,1.0
1,"Jessica M. Esparza, Eileen OÕToole, Linya Li, ...",Katanin Localization Requires Triplet Microtub...,"PLOS ONE, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053940,2013.0,01January,NaN,1.0,NaN,NaN,duplicated colony strip,1,01/15/2016,NaN,1.0,NaN,1.0
2,"Sreedevi Avasarala, Fangfang Zhang, Guangliang...",Curcumin Modulates the Inflammatory Response a...,"PLOS ONE, February 2013 | Volume 8 | Issue 2 |...",10.1371/journal.pone.0057285,2013.0,02February,NaN,NaN,1.0,NaN,reuse of one lane in different blot experiment,1,8/4/2015,NaN,1.0,NaN,1.0
3,"Rounak Nassirpour, Pramod P. Mehta, Sangita M....",miR-221 Promotes Tumorigenesis in Human Triple...,"PLOS ONE, April 2013 | Volume 8 | Issue 4 | e6...",10.1371/journal.pone.0062170,2013.0,04April,1.0,NaN,NaN,1,Duplicated / mirrored bands in blot,1,4/10/2017,1.0,NaN,NaN,1.0
4,"Jianuo Liu, Peng Xu, Cory Collins, Han Liu, J...",HIV-1 Tat Protein Increases Microglial Outward...,"PLOS ONE, May 2013 | Volume 8 | Issue 5 | e649...",10.1371/journal.pone.0064904,2013.0,05May,NaN,1.0,NaN,NaN,multiple duplicated microscopy images,1,9/19/2014,NaN,1.0,NaN,1.0


In [3]:
header=headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36'}

In [4]:
bik['Title'][0]

'Chronic Morphine Treatment Attenuates Cell Growth of Human BT474 Breast Cancer Cells by Rearrangement of the ErbB Signalling Network'

In [5]:
bik['DOI'][0]

'10.1371/journal.pone.0053510'

In [6]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
driver=webdriver.Chrome()
driver.get("https://www.researchgate.net/login")
# Type the user and password
driver.find_element(By.ID, "input-login").send_keys(user_email)
driver.find_element(By.ID, "input-password").send_keys(user_pwd)
# Actually log in
driver.find_element(By.CLASS_NAME, "nova-legacy-c-button__label").find_element(By.XPATH, "./..").click()
#os.mkdir('pdfs')
for i in range(len(bik)): 
    curr_page = 'https://www.researchgate.net/search/publication?q='+bik['DOI'][i]
    driver.get(curr_page)
    bs_test=bs(driver.page_source,'html.parser')
    if bs_test.find('a',{'class':'nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--size-m nova-legacy-c-button--color-blue nova-legacy-c-button--theme-solid nova-legacy-c-button--width-auto'})!=None:
        url=bs_test.find('a',{'class':'nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--size-m nova-legacy-c-button--color-blue nova-legacy-c-button--theme-solid nova-legacy-c-button--width-auto'})['href']
        pdf_url='https://www.researchgate.net/'+url
        local_file='pdfs/'+bik['Title'][i].replace('/','')[:100]+'.pdf'
        with open(local_file,'wb+') as f:
            f.write(requests.get(pdf_url).content)
    else:
        if bs_new.find('div',{'class':'dload-pdf'})!=None:
            new_page='https://journals.plos.org/plosone/article?id='+bik['DOI'][i]
            new_page=requests.get(new_page)
            bs_new=bs(new_page.text,'html.parser')
            url=bs_new.find('div',{'class':'dload-pdf'}).find('a')['href']
            pdf_url='https://journals.plos.org/'+url
            local_file='pdfs/'+bik['Title'][i].replace('/','')[:100]+'.pdf'
            with open(local_file,'wb+') as f:
                f.write(requests.get(pdf_url).content)
        else:
            new_pubmed_url='https://pubmed.ncbi.nlm.nih.gov/?term='+bik['DOI'][i]
            new_pubmed=requests.get(new_pubmed_url)
            bs_pubmed=bs(new_pubmed.text,'html.parser')
            if bs_pubmed.find('a',{'data-ga-action':'DOI'})!=None:
                turning_url=bs_pubmed.find('a',{'data-ga-action':'DOI'})['href']
                driver.get(turning_url)
                bs_turning_url=bs(driver.page_source,'html.parser')
                if bs_turning_url.find('div',{'class':'info-panel__formats info-panel__item'})!=None:
                    pubmed_url=bs_turning_url.find('div',{'class':'info-panel__formats info-panel__item'}).find('a')['href']
                    pdf_pubmed_url='https://journals.asm.org/'+pubmed_url
                    local_file='pdfs/'+bik['Title'][i].replace('/','')[:100]+'.pdf'
                    with open(local_file,'wb+') as f:
                        f.write(requests.get(pdf_pubmed_url).content)
                else:
                    curr_page = 'https://www.researchgate.net/search/publication?q='+bik['Title'][i]
                    driver.get(curr_page)
                    bs_title=bs(driver.page_source,'html.parser')
                    if bs_title.find('a',{'class':'nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--size-s nova-legacy-c-button--color-blue nova-legacy-c-button--theme-ghost nova-legacy-c-button--width-auto'})!=None:
                        url=bs_title.find('a',{'class':'nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--size-s nova-legacy-c-button--color-blue nova-legacy-c-button--theme-ghost nova-legacy-c-button--width-auto'})['href']
                        pdf_url='https://www.researchgate.net/'+url
                        local_file='pdfs/'+bik['Title'][i].replace('/','')[:100]+'.pdf'
                        with open(local_file,'wb+') as f:
                            f.write(requests.get(pdf_url).content)
                    else:
                        local_file='pdfs/'+bik['Title'][i].replace('/','')[:100]+'.pdf'
                        with open(local_file,'wb+') as f:
                            f.write(requests.get('https://onlinelibrary.wiley.com/doi/epdf/'+bik['DOI'][i]).content)
            else:
                curr_page = 'https://www.researchgate.net/search/publication?q='+bik['Title'][i]
                driver.get(curr_page)
                bs_title=bs(driver.page_source,'html.parser')
                if bs_title.find('a',{'class':'nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--size-s nova-legacy-c-button--color-blue nova-legacy-c-button--theme-ghost nova-legacy-c-button--width-auto'}) == None:
                    continue
                else:
                    url=bs_title.find('a',{'class':'nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--size-s nova-legacy-c-button--color-blue nova-legacy-c-button--theme-ghost nova-legacy-c-button--width-auto'})['href']
                    pdf_url='https://www.researchgate.net/'+url
                    local_file='pdfs/'+bik['Title'][i].replace('/','')[:100]+'.pdf'
                    with open(local_file,'wb+') as f:
                        f.write(requests.get(pdf_url).content)
                
    print('finish'+str(i))
    #driver.find_element(By.LINK_TEXT, "Download").click()

In [ ]:
local_file

In [ ]:
pdf_url

In [ ]:
with open('ddd.pdf','wb') as f:
    f.write(requests.get(pdf_url+'.pdf').content)

In [ ]:
requests.get(pdf_url,headers=headers)

In [10]:
driver=webdriver.Chrome()
driver.get("https://www.researchgate.net/login")
# Type the user and password
driver.find_element(By.ID, "input-login").send_keys(user_email)
driver.find_element(By.ID, "input-password").send_keys(user_pwd)
# Actually log in
driver.find_element(By.CLASS_NAME, "nova-legacy-c-button__label").find_element(By.XPATH, "./..").click()
#os.mkdir('pdfs')
for i in range(165,len(bik)): 
    curr_page = 'https://www.researchgate.net/search/publication?q='+bik['DOI'][i]
    driver.get(curr_page)
    bs_test=bs(driver.page_source,'html.parser')
    if bs_test.find('a',{'class':'nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--size-m nova-legacy-c-button--color-blue nova-legacy-c-button--theme-solid nova-legacy-c-button--width-auto'})!=None:
        driver.find_element(By.LINK_TEXT, "Download").click()
    else:
        curr_page = 'https://www.researchgate.net/search/publication?q='+bik['Title'][i]
        driver.get(curr_page)
        bs_title=bs(driver.page_source,'html.parser')
        if bs_title.find('a',{'class':'nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--size-s nova-legacy-c-button--color-blue nova-legacy-c-button--theme-ghost nova-legacy-c-button--width-auto'})!=None:
            url=bs_title.find('a',{'class':'nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--size-s nova-legacy-c-button--color-blue nova-legacy-c-button--theme-ghost nova-legacy-c-button--width-auto'})['href']
            pdf_url='https://www.researchgate.net/'+url
            driver.find_element(By.LINK_TEXT, "Download").click()
    print('finish '+str(i))

finish 0
finish 1
finish 2
finish 3
finish 4
finish 5
finish 6
finish 7
finish 8
finish 9
finish 10
finish 11
finish 12
finish 13
finish 14
finish 15
finish 16
finish 17
finish 18
finish 19
finish 20
finish 21
finish 22
finish 23
finish 24
finish 25
finish 26
finish 27
finish 28
finish 29
finish 30
finish 31
finish 32
finish 33
finish 34
finish 35
finish 36
finish 37
finish 38
finish 39
finish 40
finish 41
finish 42
finish 43
finish 44
finish 45
finish 46
finish 47
finish 48
finish 49
finish 50
finish 51
finish 52
finish 53
finish 54
finish 55
finish 56
finish 57
finish 58
finish 59
finish 60
finish 61
finish 62
finish 63
finish 64
finish 65
finish 66
finish 67
finish 68
finish 69
finish 70
finish 71
finish 72
finish 73
finish 74
finish 75
finish 76
finish 77
finish 78
finish 79
finish 80
finish 81
finish 82
finish 83
finish 84
finish 85
finish 86
finish 87
finish 88
finish 89
finish 90
finish 91
finish 92
finish 93
finish 94
finish 95
finish 96
finish 97
finish 98
finish 99
finish 100

KeyboardInterrupt: 

In [11]:
bik['Title'][164]

'Effect of sodium butyrate on lung vascular TNFSF15 (TL1A) expression: Differential expression patterns in pulmonary artery and microvascular endothelial cells'

In [12]:
bik['DOI'][164]

'10.1016/j.cyto.2008.12.013'

In [ ]:
"https://onlinelibrary.wiley.com/doi/pdfdirect/"+bik['DOI'][164]+"?download=true"

In [9]:
driver=webdriver.Chrome()
driver.get("https://www.researchgate.net/login")
# Type the user and password
driver.find_element(By.ID, "input-login").send_keys(user_email)
driver.find_element(By.ID, "input-password").send_keys(user_pwd)
# Actually log in
driver.find_element(By.CLASS_NAME, "nova-legacy-c-button__label").find_element(By.XPATH, "./..").click()
#os.mkdir('pdfs')
researchgate_notfound=[]
for i in range(len(bik)): 
    try:
        curr_page = 'https://www.researchgate.net/search/publication?q='+bik['DOI'][i]
        driver.get(curr_page)
        bs_test=bs(driver.page_source,'html.parser')
        driver.find_element(By.LINK_TEXT, "Download").click()
        print('finish '+str(i))
    except:
        researchgate_notfound.append(i)
        print('still need found'+str(i))

finish 0
finish 1
finish 2
finish 3
finish 4
finish 5
finish 6
finish 7
finish 8
finish 9
finish 10
finish 11
finish 12
finish 13
finish 14
finish 15
finish 16
finish 17
finish 18
finish 19
finish 20
finish 21
finish 22
finish 23
finish 24
still need found25
finish 26
finish 27
finish 28
finish 29
finish 30
finish 31
finish 32
finish 33
finish 34
finish 35
finish 36
finish 37
finish 38
finish 39
finish 40
finish 41
finish 42
finish 43
finish 44
finish 45
finish 46
finish 47
finish 48
finish 49
finish 50
finish 51
finish 52
finish 53
finish 54
finish 55
finish 56
finish 57
finish 58
still need found59
finish 60
finish 61
finish 62
finish 63
finish 64
finish 65
finish 66
finish 67
finish 68
finish 69
finish 70
finish 71
finish 72
finish 73
finish 74
finish 75
still need found76
finish 77
finish 78
still need found79
finish 80
finish 81
finish 82
still need found83
finish 84
finish 85
finish 86
finish 87
finish 88
still need found89
finish 90
finish 91
finish 92
finish 93
finish 94
finish

In [12]:
'https://uosc.primo.exlibrisgroup.com/discovery/search?query=any,contains,%22'+bik['DOI'][195]+'%22&tab=Everything&search_scope=MyInst_and_CI&vid=01USC_INST:01USC&lang=en&offset=0'

'https://uosc.primo.exlibrisgroup.com/discovery/search?query=any,contains,%2210.3892/ijo.2012.1617%22&tab=Everything&search_scope=MyInst_and_CI&vid=01USC_INST:01USC&lang=en&offset=0'

In [112]:
usc_url=[]
usc_notfound=[]
driver=webdriver.Chrome()
for i in (researchgate_notfound):
    curr_usc_page='https://uosc.primo.exlibrisgroup.com/discovery/search?query=any,contains,%22'+bik['DOI'][i]+'%22&tab=Everything&search_scope=MyInst_and_CI&vid=01USC_INST:01USC&lang=en&offset=0'
    driver.get(curr_usc_page)
    driver.implicitly_wait(10)
    time.sleep(10)
    bs_usc=bs(driver.page_source,'html.parser')
    if bs_usc.find('a',{'class':'browzine-direct-to-pdf-link'})!=None:
        usc_url.append(bs_usc.find('a',{'class':'browzine-direct-to-pdf-link'})['href'])
        driver.implicitly_wait(10)
        driver.find_element(By.CLASS_NAME, "browzine-direct-to-pdf-link").find_element(By.XPATH, "./..").click()
        print('usc found '+str(i))
    else:
        usc_notfound.append(i)
        print('usc not found'+str(i))

usc found 25
usc found 59
usc found 76
usc not found79
usc found 83
usc found 89
usc found 109
usc found 110
usc found 111
usc found 112
usc found 113
usc found 114
usc found 115
usc found 116
usc found 117
usc found 119
usc found 120
usc found 121
usc found 122
usc found 123
usc found 124
usc found 125
usc not found135
usc not found136
usc found 138
usc found 139
usc found 140
usc found 141
usc found 142
usc found 143
usc not found145
usc found 146
usc found 147
usc found 148
usc found 149
usc found 150
usc not found151
usc found 152
usc found 153
usc not found154
usc not found155
usc not found156
usc not found157
usc found 158
usc not found159
usc not found160
usc found 161
usc not found162
usc found 163
usc not found164
usc found 165
usc not found166
usc found 167
usc not found168
usc not found169
usc found 170
usc found 171
usc found 172
usc not found173
usc found 174
usc found 175
usc found 176
usc found 177
usc found 180
usc found 181
usc found 183
usc found 188
usc found 190
usc

In [115]:
len(usc_notfound)

17

In [121]:
usc_notfound_DOI=[]
usc_notfound_title=[]
for i in usc_notfound:
    usc_notfound_DOI.append(bik['DOI'][i])
    usc_notfound_title.append(bik['Title'][i])

In [117]:
usc_notfound_DOI#这些实在是找不到出处了

['10.1128/IAI.01769-14',
 '10.1016/S0169-5002(01)00212-4',
 '10.1016/S0169-5002(03)00239-3',
 '10.1016/j.jaut.2004.10.003',
 '10.1016/j.cyto.2004.12.015',
 '10.1016/j.cyto.2007.01.002',
 '10.1016/j.cyto.2006.12.003',
 '10.1016/j.cyto.2005.12.007',
 '10.1016/j.cyto.2007.04.003',
 '10.1016/j.cyto.2008.06.003',
 '10.1016/j.cyto.2008.01.001',
 '10.1016/j.cyto.2009.07.004',
 '10.1016/j.cyto.2008.12.013',
 '10.1016/j.cyto.2009.11.020',
 '10.1016/j.cyto.2011.08.017',
 '10.1016/j.cyto.2011.02.016',
 '10.1016/j.cyto.2012.08.025']

In [118]:
'https://onlinelibrary.wiley.com/doi/pdfdirect/'+usc_notfound_DOI[0]+'?download=true'

'https://onlinelibrary.wiley.com/doi/pdfdirect/10.1128/IAI.01769-14?download=true'

In [122]:
usc_notfound_title

['Two Functional Type VI Secretion Systems in Avian Pathogenic Escherichia coli Are Involved in Different Pathogenic Pathways',
 'Low titre autoantibodies against recoverin in sera of patients with small cell lung cancer but without a loss of vision',
 'Antirecoverin autoantibodies in the patient with non-small cell lung cancer but without cancer-associated retinopathy',
 'Differential regulation of Cu, Zn- and Mn-superoxide dismutases by retinoic acid in normal and psoriatic human fibroblasts',
 'cDNA cloning and functional analysis of goose interleukin-2 ',
 'Time-dependent onset of Interferon-a2b-induced apoptosis in isolated hepatocytes from preneoplastic rat livers',
 'STAT1 and STAT3 phosphorylation by porins are independent of JAKs but are dependent on MAPK pathway and plays a role in U937 cells production of interleukin-6',
 'Tumor necrosis factor alpha down-regulates the Na+-K+ ATPase and the Na+-K+2Cl- cotransporter in the kidney cortex and medulla',
 'Upregulation of tumor n